# モデルの定式化

## モデルとは何か?  なぜ直接ソルバーを使わないのか?

![Why not use solvers directly?](./assets/creating_models_01.png)

`jijmodeling`は、人間が読みやすい数学モデルをコンピュータが読み取れるデータ形式に変換する「モデラー」ライブラリです。最適化問題にはいくつかの種類があり、それぞれに対応する問題固有のソルバーが存在しますが、これらのソルバーは特定のデータ形式のみを受け入れます。そのため、特定の問題インスタンスに固有のデータを組み込む必要があります。`jijmodeling`を使用すると、最適化モデルを単一の数学的な方法で記述し、その後でソルバーやインスタンス固有の詳細に適応させることができます。

## 定式化の例

$N$個の実数係数$d_n$を持つ単純なバイナリ線形最小化問題を考えます：

$$
\min \sum_{n=0}^{N−1}d_n x_n, \\
\text{s.t.} \space \sum_{n=0}^{N-1} x_n = 1, \\
\text{where} \space x_n \in \{0,1\} \space \forall n \in [0, N-1]
$$

これは`jijmodeling`の基本的な使用方法を示す例です。以下を含みます：

- 決定変数$x_n$とパラメータ$N$および$d_n$を定義する
- 目的関数を$\sum_{n=0}^{N-1} d_n x_n$の最小化として設定する
- 等式制約$\sum_{n=0}^{N-1} x_n = 1$を追加する

より実践的で包括的な例は、JijZeptのドキュメントサイトの[Learn](https://www.documentation.jijzept.com/docs/category/learn)で確認することができます。

## `Problem`オブジェクトを作る

コードを使って話し始めましょう。まず、jijmodelingをインポートする必要があります。

In [ ]:
import jijmodeling as jm
jm.__version__   # 1.8.0

jijmodelingのバージョンがこのドキュメントに対応していることを確認することを強くお勧めします。その後、以下のコードを実行してください。

`Problem`の役割は、数学モデルをPythonオブジェクトとして表現することです。

In [ ]:
# Define parameters
d = jm.Placeholder("d", ndim=1)
N = d.len_at(0, latex="N")

# Define decision variables
x = jm.BinaryVar("x", shape=(N,))

# Index for calc sum
n = jm.Element('n', belong_to=(0, N))

# create problem instance 
problem = jm.Problem('my_first_problem')
# Set objective
problem += jm.sum(n, d[n] * x[n])
# Set constraint
problem += jm.Constraint("onehot", jm.sum(n, x[n]) == 1)

# See problem on REPL/Jupyter
problem

上記の数学モデルに対応するコードの部分を見つけることができると考えています。このドキュメントでは、各タイプの概念とその操作についてもう少し詳しく説明しています。

:::{admonition} [Jupyter](https://jupyter.org/)環境
:class: note

Jupyterや関連する環境では、以下のようにして`Problem`オブジェクトの内容を表示できます。これにより、モデルを対話的にデバッグするのに役立ちます。

![Jupyter example](./assets/creating_models_02.png)

:::

## 決定変数とパラメータ

上記のモデルには2種類の「変数」があります：決定変数とパラメータです。jijmodelingでは、オブジェクトを宣言する際に使用するクラスによってこれが決まります。

![Decision variables and Parameters](./assets/creating_models_03.png) <!-- https://docs.google.com/drawings/d/1MZwFljVOV0XZpTB_vYpOtd7_2jqn3xM3Wx1IAwB55EM/edit?usp=sharing -->

- $x_n$の値は問題を解くことで決定されます。これらは「決定変数」と呼ばれます。
    - この問題では、$x_n \in \{0, 1\}$のバイナリ変数を`BinaryVar`で表現しています。決定変数を定義するために選択できる他のタイプには、`IntegerVar`や`ContinuousVar`があります。
    - [変数の種類と境界](./types_and_bounds.ipynb)で異なる変数タイプについてさらに詳しく説明します。
- $N$と$d$の値はユーザーによって指定される「空欄」です。
    - この問題は$N$と$d$によってパラメータ化されていると言います。
    - これらの実際の数値は`Problem`オブジェクト内で指定されていません。
    - これらは問題の「インスタンスデータ」の一部と見なすことができます。特定のインスタンスは異なる値を持ちますが、特定の値に依存しない方法でモデルを記述することができます。
    - ほとんどのパラメータは、上記のコードのようにdとして明示的に定義された`Placeholder`オブジェクトで表されます。
    - Nは$d$の要素数として定義しました（これは`ArrayLength`オブジェクトです）。これにより、$N$は暗黙のパラメータとなります：インスタンスを定義するために$d$を指定するだけで済みます。これにより、数学モデル内の$N$の意味がコードの明確な部分となります。

:::{admonition} オブジェクトとは何か？
:class: tip

Pythonでは、すべての値には型があります。例えば、`1`は`int`型であり、`1.0`は`float`型です。組み込み関数`type`を使用して、`type(1.0)`のように型を取得できます。ある型`A`に対して、型Aの値を「`A`オブジェクト」と呼びます。

:::

### 多次元変数

インデックスを使用できる変数を定義することができます。これは、変数の配列や行列を持つことに類似しています。$N$個の係数$d_n$と$N$個の決定変数$x_n$があると仮定し、それらを一次元の`Placeholder` `d`と一次元の`BinaryVar` `x`として記述します。`Placeholder`の場合、値の数を指定せずに一次元であることを示すだけで済みます。しかし、決定変数の場合、その数は次元数とともに指定する必要があります。ただし、その数はパラメータに関連して定義することができ、具体的な数値を使用する必要はありません。例えば、次のように：

In [ ]:
# we first define d
d = jm.Placeholder("d", ndim=1)
# and then take the size of it as N
N = d.len_at(0, latex="N")
# x is defined with this size
x = jm.BinaryVar("x", shape=(N,))

オブジェクト`N`は`ArrayLength`型であり、これは`Placeholder` `d`の要素数を表します。`len_at`に与えられた`0`パラメータは、`Placeholder`が任意の次元数を持つことができるためですが、長さを明確に定義するためには、カウントする軸を指定する必要があります。

:::{note} インデックス付けと総和については、[次のページ](./expressions.ipynb)でさらに詳しく説明します。 :::

## 目的関数

次に、$\sum_{n=0}^{N-1} d_n x_n$を問題の最小化対象として設定したいと考えます。しかし、$N$はまだ固定されていないため、Pythonで`for`ループを書くことができません。では、どのようにしてこれらを合計するのでしょうか？

この問題を解決するためにjm.sumが存在します：

In [ ]:
n = jm.Element('n', belong_to=(0, N))
sum_dx = jm.sum(n, d[n] * x[n])

`Element`は、ある範囲内のインデックスに対応する新しい変数タイプです。数学では通常、

> 与えられた$n \in [0, N-1]$に対して、$d \in \mathbb{R}^N$の$n$番目の要素を取ります。

`jijmodeling`では、これは$n \in [0, N-1]$に対応する`Element`オブジェクト`n`と、$d_n$に対応する表現`d[n]`で表されます。`Element`オブジェクトに有効なインデックス範囲が格納されていることを確認してください。`sum`はインデックスとして要素nを取り、表現`d[n] * x[n]`を取り、$\sum_n d_n x_n$に対応する新しい表現を返します。

:::{note}
「表現」については[次のページ](./expressions.ipynb)で詳しく説明します。
:::

ここで、`Problem`インスタンスを作成し、$\sum_n d_n x_n$を目的関数として設定できます：

In [ ]:
problem = jm.Problem('my_first_problem')
problem += sum_dx

目的関数を最大化したい場合は、`Problem`を構築する際に`sense`パラメータを設定できます。

In [ ]:
problem = jm.Problem('my_first_problem', sense=jm.ProblemSense.MAXIMIZE)

## 等式制約

最後に、等式制約に対応する`Constraint`オブジェクトを作成しましょう。

$$
\sum_{n=0}^{N-1} x_n = 1
$$

上記で説明した`sum`表現を使用して、この制約を表現として記述できます。

In [ ]:
jm.sum(n, x[n]) == 1

通常のPythonの型とは異なり、`==`がbool値を返すのに対し、`jijmodeling`の表現における`==`は等式比較を表す新しい表現を返します。`Constraint`オブジェクトは名前と有効な比較表現（`==`、`<=`、または`>=`を使用）で作成されます。これを`problem`に追加することができます。

In [ ]:
problem += jm.Constraint("onehot", jm.sum(n, x[n]) == 1)

:::{note}
このトピックについては[制約条件とペナルティ](./constraint_and_penalty.ipynb)ページでさらに詳しく説明します。
:::